In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

path = 'https://github.com/oregon-data-science/DSCI101/raw/main/data/'

'imports OK'

# Table manipulation summary, demonstrated using the previously seen Census database

## 1. load the table

In [ ]:
full = Table.read_table(path + 'nc-est2014-agesex-res.csv')
full

In [ ]:
Table.num_rows?

## 2. Note the number of columns, rows, and column labels

In [ ]:
print("Number of columns:", full.num_columns)
print("Number of rows:", full.num_rows)
print("Column labels:", full.labels)

## 3. Use select() or drop() to yield a table with the columns you desire

In [ ]:
partial = full.select('SEX', 'AGE', 'POPESTIMATE2010', 'POPESTIMATE2014')
partial

## 4. Make things easier to read

In [ ]:
simple = partial.relabeled(2, '2010').relabeled(3, '2014')
simple

In [ ]:
partial.relabeled?

## 5. Determine unique values in some of the columns

In [ ]:
print("unique values in SEX column:", np.unique(simple.column('SEX')))
print("unique values in AGE column:\n", np.unique(simple.column('AGE')))

In [ ]:
 np.unique?

## 6. Use where() to keep only the rows you want - several examples below

In [ ]:
simple.where('SEX', 1)

In [ ]:
simple.sort('AGE').where('SEX', are.not_equal_to(0))

In [ ]:
simple.where('SEX', 0).where('AGE', are.between(10, 20))

In [ ]:
simple.where('SEX', 0).drop('SEX').where('AGE', are.between(10, 20)).plot('AGE')

## 7. Use column() to extract an array to manipulate - for example, do the following to determine the average age in 2010 and 2014

In [ ]:
total_by_age = simple.where('SEX', 0).where('AGE', are.below(999))
total_by_age

In [ ]:
## We can only do math on arrays (vs objects of type table, even if they are only a column)
## How could I calculate the average age in 2010? 
sum_ages = sum(total_by_age.column('AGE') * total_by_age.column('2010'))
sum_ages/sum(total_by_age.column('2010'))

## 8. Make computing the average age a function

In [ ]:
def average_age(age_array, num_array):
    return sum(age_array * num_array) / sum(num_array)

In [ ]:
total_by_age = simple.where('SEX', 0).where('AGE', are.below(999))

## in 2010 
round(average_age(total_by_age.column('AGE'), total_by_age.column('2010')), 2)

In [ ]:
## in 2014
round(average_age(total_by_age.column('AGE'), total_by_age.column('2014')), 2)

In [ ]:
## Why make functions? 

## 9. Add a column to help you select the appropriate rows

Suppose we only want rows from `simple` where females outnumber males in 2014

In [ ]:
# what do we need to do?
## end goal: Create a table where we can compare the number of females and males, 
## then subset to the rows where number females > number males

# a. Clean up main table of unwanted rows: drop rows where AGE == 999
# b. Get the males: extract rows where SEX == 1, assigned to males
# c. Get the Females: extract rows where SEX == 2, assigned to females
# d. Combine males and females for comparison: create a new table with three columns: AGE, MALE, FEMALE
# e. Make the comparison: compute FEMALE/MALE ratio
# f. Subsect the table: extract rows where RATIO > 1.0, dropping RATIO from the table

In [ ]:
clean_census = simple.where('AGE', are.below(999))    # a

In [ ]:
males = clean_census.where('SEX', 1)                  # b

In [ ]:
females = clean_census.where('SEX', 2)                # c

In [ ]:
comp_males_females = Table().with_columns(
    'AGE', males.column('AGE'),
    'MALE', males.column('2014'),
    'FEMALE', females.column('2014'),
)                                           # d

In [ ]:
comp_males_females = comp_males_females.with_column('RATIO',
                    comp_males_females.column("FEMALE")/comp_males_females.column("MALE"))
comp_males_females # e

In [ ]:
females_outnumber_males = comp_males_females.where('RATIO', are.above(1.0)).drop('RATIO') # g
females_outnumber_males.show()